In [13]:
# Warning!!!: On all those variables, don't remove the starting/ending quotes when doing copy/paste
# Enter the main namespace name of this lab. It should be in the form xraylab-{guid}, like xraylab-f2d5
namespace = 'xraylab-0001'

# Enter you bucket base name. That's the same one you have put in the config map. It should be identical as your namespace if you have followed instructions.
bucket_base_name = 'xraylab-0001'

# Enter you Access and Secret keys. They're the ones that were displayed in the instructions.
aws_access_key_id = '02JMAK69UEON1JGYKSN9'
aws_secret_access_key = 'fedp7q3LodO2hiGQyP2SVJSTDwL6BfScMjBLs0ey'

# Do not change this value, this is the internal location for the RGW
endpoint_url = 'http://rook-ceph-rgw-s3a.openshift-storage.svc.cluster.local'

attributes = {}
attributes['push-endpoint'] = 'kafka://my-cluster-kafka-bootstrap.'+namespace+':9092'
attributes['kafka-ack-level'] = 'broker'

In [14]:
pip install boto3

You should consider upgrading via the '/opt/app-root/bin/python3.6 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [15]:
import boto3
import json
import botocore
import argparse

In [16]:
s3 = boto3.client('s3',
                endpoint_url = endpoint_url,
                aws_access_key_id = aws_access_key_id,
                aws_secret_access_key = aws_secret_access_key,
                region_name = 'default',
                config=botocore.client.Config(signature_version = 's3'))

sns = boto3.client('sns', 
                endpoint_url = endpoint_url, 
                aws_access_key_id = aws_access_key_id,
                aws_secret_access_key= aws_secret_access_key,
                region_name='default', 
                config=botocore.client.Config(signature_version = 's3'))

In [17]:
for bucket in s3.list_buckets()['Buckets']:
    print(bucket['Name'])

xraylab-0001
xraylab-0001-anonymized
xraylab-0001-kogito-inbound
xraylab-0001-kogito-outbound
xraylab-0001-processed


In [18]:
for bucket in s3.list_buckets()['Buckets']:
    bucket_policy = {
                      "Version":"2012-10-17",
                      "Statement":[
                        {
                          "Sid":"AddPerm",
                          "Effect":"Allow",
                          "Principal": "*",
                          "Action":["s3:GetObject"],
                          "Resource":["arn:aws:s3:::{0}/*".format(bucket['Name'])]
                        }
                      ]
                    }
    bucket_policy = json.dumps(bucket_policy)
    s3.put_bucket_policy(Bucket=bucket['Name'], Policy=bucket_policy)

In [21]:
def create_topic(topic):
    topic_arn = sns.create_topic(Name=topic, Attributes=attributes)['TopicArn']
    return topic_arn

In [22]:
create_topic('xray-images-anonymized')

'arn:aws:sns:s3a::xray-images-anonymized'

In [23]:
sns.list_topics()

{'Topics': [{'TopicArn': 'arn:aws:sns:s3a::xray-images'},
  {'TopicArn': 'arn:aws:sns:s3a::xray-images-anonymized'},
  {'TopicArn': 'arn:aws:sns:s3a::xray-images'}],
 'ResponseMetadata': {'RequestId': 'f089746f-485e-410a-990a-e00f621fcdc3.164817.3298',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-request-id': 'tx000000000000000000ce2-00603583c9-283d1-s3a',
   'content-type': 'application/xml',
   'content-length': '2209',
   'date': 'Tue, 23 Feb 2021 22:38:01 GMT',
   'connection': 'Keep-Alive'},
  'RetryAttempts': 0}}

In [24]:
bucket_notifications_configuration = {
            "TopicConfigurations": [
                {
                    "Id": 'xray-images-anonymized',
                    "TopicArn": 'arn:aws:sns:s3a::xray-images-anonymized',
                    "Events": ["s3:ObjectCreated:*"]
                }
            ]
        }

bucket_name = 'xraylab-0001-anonymized'
s3.put_bucket_notification_configuration(Bucket = bucket_name,
        NotificationConfiguration=bucket_notifications_configuration)

{'ResponseMetadata': {'RequestId': 'tx000000000000000000ced-0060358426-283d1-s3a',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-request-id': 'tx000000000000000000ced-0060358426-283d1-s3a',
   'content-length': '0',
   'date': 'Tue, 23 Feb 2021 22:39:34 GMT',
   'connection': 'Keep-Alive'},
  'RetryAttempts': 0}}